In [ ]:
import requests
import openai

# Set up your API keys
GOOGLE_KG_API_KEY = 'your_google_kg_api_key'
OPENAI_API_KEY = 'your_openai_api_key'

# Define the function to query Google Knowledge Graph
def query_knowledge_graph(query):
    url = "https://kgsearch.googleapis.com/v1/entities:search"
    params = {
        'query': query,
        'limit': 1,
        'indent': True,
        'key': GOOGLE_KG_API_KEY,
    }
    response = requests.get(url, params=params)
    data = response.json()
    if 'itemListElement' in data and len(data['itemListElement']) > 0:
        entity = data['itemListElement'][0]['result']
        return entity.get('name', 'Unknown'), entity.get('description', 'No description available.')
    else:
        return "Entity not found", ""

# Define the function to use GPT-3 for generating responses
def generate_gpt_response(question):
    openai.api_key = OPENAI_API_KEY
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=question,
        max_tokens=100
    )
    return response.choices[0].text.strip()

# Main function to integrate the LLM and Knowledge Graph
def knowledge_graph_llm_pipeline(query):
    # Step 1: Query the knowledge graph
    entity_name, entity_description = query_knowledge_graph(query)
    
    # Step 2: Generate a question for GPT-3 using the knowledge graph info
    if entity_name != "Entity not found":
        gpt_question = f"What can you tell me about {entity_name}? {entity_description}"
    else:
        gpt_question = f"Tell me more about {query} in general."

    # Step 3: Get a response from the LLM
    gpt_response = generate_gpt_response(gpt_question)
    
    return {
        'Entity': entity_name,
        'Knowledge Graph Info': entity_description,
        'LLM Response': gpt_response
    }

# Example usage:
query = "Albert Einstein"
result = knowledge_graph_llm_pipeline(query)
print("Knowledge Graph Entity:", result['Entity'])
print("Knowledge Graph Info:", result['Knowledge Graph Info'])
print("LLM Response:", result['LLM Response'])
